In [2]:
%%capture
!pip install gradio

In [3]:
%%capture
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [4]:
import pandas as pd
import gradio as gr
import math as m
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt

## Training Parameters

>



In [5]:

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', device='cpu')  
model.conf = 0.4  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = True  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.classes = [0,32]  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
model.max_det = 4  # maximum number of detections per image
model.amp = False  # Automatic Mixed Precision (AMP) inference


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-2-11 torch 1.10.0+cu111 CPU



  0%|          | 0.00/14.0M [00:00<?, ?B/s]

Fusing layers... 


Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


## Utility fuctions for calcilations

In [6]:
def calculate_centres(df):
  # gets dataframe and returns it with mid points for bounding boxes

  df["xcentre"] = (df.xmin + df.xmin)/2
  df["ycentre"] = (df.ymin + df.ymax)/2

  return df

In [7]:
def calculate_distance(data):
  # gets dataframe and returns the distance to the ball for each class

  df = data.copy()
  xball = df[df["class"]==32].xcentre.values[0]
  yball = df[df["class"]==32].ycentre.values[0]

  df["xball"] = xball
  df["yball"] = yball

  df["x_d"] = df.xball - df.xcentre
  df["y_d"] = df.yball - df.ycentre

  df["distance"] = np.sqrt(df.x_d**2 + df.y_d**2)

  return df

In [8]:
def get_second_largest(data):
  # gets dataframe and returns nearest player to the ball
  
  index = data.distance.sort_values().head(2).index[1].copy()
  df = data.iloc[index]

  return df


In [9]:
def draw_possession(df,img):
  # draws bounding boxes given the datframe output from prediction

  # cv2.rectangle(img, pt1, pt2, color, thickness, lineType, shift)
  #image = cv2.imread(img)
  box = cv2.rectangle(img, (int(df.xmin), int(df.ymin)), (int(df.xmax), int(df.ymax)), (255,0,0), 4)
  # displaying image
  #plt.imshow(box)
  #plt.show()

  return box


In [10]:
def predict(image):
  # main prediction function that gets image and returns pandas dataframe

  results = model(image, size=640)  # custom inference size
  return results.pandas().xyxy[0] #change to return both
  
def processed(image):
  
  results = predict(image)
  results = calculate_centres(results)
  processed = calculate_distance(results)
  processed = get_second_largest(processed)
  box = draw_possession(processed,image)

  return box




## Gradio Deployment

In [12]:
# launching gradio interface

iface = gr.Interface(fn = processed, inputs = "image", 
             outputs = "image",
             capture_session = True, 
             title="Who Has the Ball?")
             
iface.launch(debug = False)

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:224: UserWarning: The `capture_session` parameter in the `Interface` is deprecated and may be removed in the future.
  "The `capture_session` parameter in the `Interface`"


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`


Connected (version 2.0, client OpenSSH_7.6p1)
Authentication (publickey) successful!


Running on public URL: https://51227.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f03d3633310>,
 'http://127.0.0.1:7861/',
 'https://51227.gradio.app')